# Sea Ice Concentration

From the NSIDC Sea Ice Monthly historical - calculate climatogy and averages for sea ice conc and extent

In [1]:
#directory paths
_work_dir='/g/data/jk72/as2285/miz/'
_data_dir='/g/data/jk72/MIZ/'

In [2]:
#some constants
CLIMAT_DATES=[1981,2010]
EAST_ANT_LONS=[71,160] #longitudes for east Antartica (easterly)
YEAR=2021
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


In [3]:
#useful py libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as ticker
import odc.geo.xr

In [4]:
# import local functions
import sys
sys.path.append(_work_dir)
from utils.sea_ice_conc import sea_ice_conc

%run {_work_dir}utils/plot_tools.ipynb

# Open the dataset

Using the NSIDC Sea Ice Concentration gridded data set, historical monthly averages of sea ice concentration

In [5]:
hist_cdr_xr = xr.open_dataset(
    f'{_data_dir}/NSIDC/G02202_V4/seaice_conc_monthly_sh_197811_202112_v04r00.nc',
    #chunks='auto'
).swap_dims(
    {'tdim':'time', 'x':'xgrid','y':'ygrid'}
).rename(
    {'cdr_seaice_conc_monthly':'cdr_seaice_conc','xgrid':'x','ygrid':'y'}
)

We are going to compare this to the current (2022) conditions:

In [6]:
daily_files = ! ls -d {_data_dir}/NSIDC/G10016_V2/daily/*.nc

In [7]:
daily_da=xr.concat(
    [
        xr.open_dataset(
            iFile,
        ).swap_dims(
            {'tdim':'time', 'x':'xgrid','y':'ygrid'}
        ).rename(
            {'xgrid':'x','ygrid':'y'}
        ) for iFile in daily_files
    ], 
    'time'
)

#we could resample to monlthy, but daily data actually works fine
current_cdr_xr=daily_da#.resample(time='M',label='left', loffset='1D').mean('time')

Put the historical monthly data, and the near real time data in one array:

In [8]:
# If the two datasets have gone out of sync, raise an error.
# This would be amiguous, the historical (released data) and the current (near-real time data) 
# are covering the same times and we don't know which one to use.
if hist_cdr_xr.time_coverage_end>current_cdr_xr.time_coverage_start:
    raise RuntimeError("Times in historical data overlap with near-real time data")

cdr_ds=xr.concat(
    [hist_cdr_xr.cdr_seaice_conc, current_cdr_xr.cdr_seaice_conc], #this is the only data field we use
    'time'
)

#merge the long and lat back in for convenience
cdr_xr=xr.merge(
    [
        cdr_ds,
        hist_cdr_xr.longitude,
        hist_cdr_xr.latitude]
)

cdr_xr

<xarray.Dataset>
Dimensions:          (time: 694, x: 316, y: 332)
Coordinates:
  * time             (time) datetime64[ns] 1978-11-01 1978-12-01 ... 2022-06-25
  * x                (x) float32 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y                (y) float32 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
Data variables:
    cdr_seaice_conc  (time, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    longitude        (y, x) float32 ...
    latitude         (y, x) float32 ...
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc_monthly qa_of_cdr_seaice_c...
    valid_range:          [  0 100]
    cell_methods:         tdim: mean

Annoyingly, the area of each grid cell is provided seperately:

In [9]:
datFile=open('/g/data/jk72/MIZ/NSIDC/pss25area_v3.dat', 'rb')
#pss25area_v3.dat: 316 columns x 332 rows
areasDmNd=np.fromfile(datFile, dtype=np.int32).reshape([332,316])

#Divide by 1000 to get km2
areasKmNd=areasDmNd/1000

# Climatologies for sea ice extent and area

Climatologies for extent and area of sea ice. 

Extent is the total area of the ocean where the concetration of sea ice is estimated to be greater than 15%

Area is the area of the sea ice only (smaller than extent). Concentrations less than 15% were discarded prior to calculating this area

In [10]:
ant_conc=sea_ice_conc(cdr_xr.cdr_seaice_conc, areasKmNd ) #local class

In [11]:
ant_conc.calc_extent()


In [12]:
ant_conc.calc_gridded_anoms()

# corners of the set

In [13]:
(x1, y1), _, (x2, y2) = odc.geo.xr.assign_crs(ant_conc.anoms_da, "epsg:3976").odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
bounds = [[y1, x1], [y2, x2]]

In [14]:
bounds

[[-39.22994121216445, -42.240892341379734], [-41.44603897171838, 135.0]]

# Lots of PNGs for a tracker 

Monthly mean concentrations (1981-2010)

In [17]:
START_YEAR='2018'

In [20]:
datetimes_xr=ant_conc.monthly_da.sel(time=slice(START_YEAR,'2050')).time

In [34]:
for iTime in datetimes_xr:

    plt.figure(figsize=(79,83), dpi=20, frameon=False, facecolor=None)
    ax=plt.subplot(projection=ccrs.SouthPolarStereo())#ccrs.epsg(3031))

    #sea ice conc anoms
    toPlot_anoms=ant_conc.anoms_da.sel(time=iTime)
    plt.contourf(
        toPlot_anoms.x, 
        toPlot_anoms.y, 
        toPlot_anoms.values,
        levels=np.arange(-0.45,0.46,.1),
        extend='both',
        transform=ccrs.SouthPolarStereo(true_scale_latitude=-70),#epsg(3976),
        cmap='coolwarm_r',
    )

    plot_stipling(
        toPlot_anoms,
        2*ant_conc.conc_climat_ds.st_dev.sel(month=iTime.dt.month),
        ax
    )

    plt.savefig(f'{_work_dir}data/tracker/sea_ice_conc_anoms/nsidc_sea_ice_conc_anoms_{iTime.dt.year.values}_{iTime.dt.month.values}.png',bbox_inches='tight', transparent="True")

    plt.close()

In [27]:
for iTime in datetimes_xr:

    plt.figure(figsize=(79,83), dpi=20, frameon=False, facecolor=None)
    ax=plt.subplot(projection=ccrs.SouthPolarStereo())

    #sea ice conc anoms
    to_plot=ant_conc.monthly_da.sel(time=iTime)
    to_plot=to_plot.where((to_plot>0.15)*(to_plot<=1))
    
    plt.pcolormesh(
        to_plot.x, 
        to_plot.y, 
        to_plot.values,
        vmin=0.15,
        vmax=1,
        transform=ccrs.SouthPolarStereo(true_scale_latitude=-70),
        cmap='Blues_r',
        shading='auto'
    )

    plt.savefig(f'{_work_dir}data/tracker/sea_ice_conc/nsidc_sea_ice_conc_{iTime.dt.year.values}_{iTime.dt.month.values}.png',bbox_inches='tight', transparent="True")

    plt.close()

In [19]:
from dea_tools.spatial import subpixel_contours

ModuleNotFoundError: No module named 'geopy'

In [ ]:
from affine import Affine

In [ ]:
to_plot_da.load()

In [ ]:
lines=subpixel_contours(to_plot_da, z_values=[0,-20],#np.arange(-70,71,20),
                        crs='EPSG:3031', affine=Affine(1,0,0,0,1,0))

In [ ]:
lines

In [ ]:
lines.plot()

# Build the map

In [ ]:
for YEAR in np.arange(START_YEAR,2031,1):

In [42]:
from ipyleaflet import basemaps,projections, WMSLayer
from ipywidgets import Layout
from sidecar import Sidecar
import leafmap


In [37]:
POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

In [55]:
conc_dict={}
anoms_dict={}
for iTime in datetimes_xr:
    conc_dict[f'{iTime.dt.year.values}-{iTime.dt.month.values}']=leafmap.ImageOverlay(
        url=f'{_work_dir}data/tracker/sea_ice_conc/nsidc_sea_ice_conc_{iTime.dt.year.values}_{iTime.dt.month.values}.png', 
        bounds=bounds, 
        opacity=1,
        name='Sea Ice Concentration'
    )
    anoms_dict[f'{iTime.dt.year.values}-{iTime.dt.month.values}']=leafmap.ImageOverlay(
        url=f'{_work_dir}data/tracker/sea_ice_conc_anoms/nsidc_sea_ice_conc_anoms_{iTime.dt.year.values}_{iTime.dt.month.values}.png', 
        bounds=bounds, 
        opacity=1,
        name='Concentration Anomaly'
    )

In [49]:
spsLayout=Layout(width='800px', height='1200px')

In [61]:
m = Map(center=(-70, 135),
        zoom=1,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

m.add_wms_layer(url='http://geos.polarview.aq/geoserver/wms', layers='polarview:coastS10', format='image/png',  transparent=True, attribution='Polarview', crs=POLAR3031, name='Coastline')

m.add_time_slider(
    conc_dict
)
m.add_time_slider(
    anoms_dict,
)

m.add_wms_layer(url='http://geos.polarview.aq/geoserver/wms', layers='polarview:graticuleS', format='image/png',  transparent=True, attribution='Polarview', crs=POLAR3031, name='Graticule')


display(m)

#sc = Sidecar(title="name")
#with sc:
#display(m)

Map(center=[-70, 135], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…